# Imports and Loading Files

In [1]:
import pandas as pd
import matplotlib
import seaborn as sns
import numpy as np

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
posts = pd.read_csv("depression_posts_full_corpus_201509.csv")

/Users/danicatanquilut/anaconda3/envs/ds100/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
posts = posts[posts['author'] != '[deleted]']

In [5]:
comments = pd.read_csv("Copy of depression_comments_sub201505.csv")

# Joining Posts and Comments

In [6]:
comments.set_index('link_id', inplace=True)

In [7]:
posts.set_index('name', inplace=True)

In [9]:
posts_and_comments = posts.join(comments, how='left', lsuffix="_post", rsuffix="_comment")

In [10]:
authors_commenting_on_own = posts_and_comments[posts_and_comments['author_post']==posts_and_comments['author_comment']]

In [11]:
own_by_post = authors_commenting_on_own.groupby('name').count()

# Visualizations

In [ ]:
own_by_post.hist('domain', bins = max(own_by_post['domain']))
#plt.xlim(0, max(own_by_post['domain']))
plt.title("Number of Comments on Post by OP")
plt.xlabel("Number of Comments")
plt.ylabel("Count")
plt.xscale('log')
#plt.yscale('log')

# Feature Generation

In [15]:
features_matrix = pd.DataFrame()

In [16]:
## length of post (characters)
num_char_in_post = []
for post in posts['selftext']:
    if type(post) == float:
        num_char_in_post.append(0)
    else:
        text = post.replace(" ", "")
        num_char_in_post.append(len(text))

In [17]:
## length of post (words)
num_words_in_post = []
for post in posts['selftext']:
    if type(post) == float:
        num_words_in_post.append(0)
    else:
        num_words_in_post.append(len(post.split()))

In [18]:
## proportion of personal pronouns used
personal_pronouns = ['i', 'i\'d', 'i\'ll', 'i\'m', 'i\'ve',
                      'me', 'my','myself']
prop_personal_pronouns = []
for post in posts['selftext']:
    if type(post) == float:
        prop_personal_pronouns.append(0)
    else:
        split_text = post.split()
        num_personal = len([i for i in split_text if i in personal_pronouns])
        if num_personal == 0:
            prop_personal_pronouns.append(0)
        else:
            prop_personal_pronouns.append(float(len(split_text)/num_personal))

In [19]:
## is self post // no real way to determine type of media post contains?
##tbh this will probably not end up being helpful bc basically all text posts
is_selfpost = posts['is_self'].values

In [20]:
## average word length
average_word_length = []
for post in posts['selftext']:
    if type(post) == float:
        average_word_length.append(0)
    else:
        split_text = post.split()
        if len(split_text) == 0:
            average_word_length.append(0)
        else:
            text_lengths = [len(x) for x in split_text]
            average_word_length.append(np.mean(text_lengths))

In [21]:
features_matrix['Length_Char'] = num_char_in_post
features_matrix['Length_Words'] = num_words_in_post
features_matrix['Avg_Word_Length'] = average_word_length
features_matrix['Prop_Personal'] = prop_personal_pronouns
features_matrix['Self_Post'] = is_selfpost

In [29]:
features_matrix[:10]

,Length_Char,Length_Words,Avg_Word_Length,Prop_Personal,Self_Post
0,757,170,4.441176,24.285714,True
1,42,13,3.230769,13.000000,True
2,450,121,3.677686,24.200000,True
3,840,218,3.853211,9.909091,True
4,893,218,4.082569,27.250000,True
5,1526,385,3.911688,64.166667,True
6,181,46,3.934783,15.333333,True
7,14,4,3.500000,0.000000,True
8,545,112,4.866071,22.400000,True
9,375,77,4.792208,0.000000,True
